In [1]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from typing_extensions import Concatenate
from langchain_openai import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [2]:
llm = ChatOpenAI(
    api_key='sk-proj-gfA9WQMdhwOmo2lOMFtAT3BlbkFJ9qwC52ExKO4FTTatmUNH',
    model='gpt-4o', 
)

embedding = OpenAIEmbeddings(
    api_key='sk-proj-gfA9WQMdhwOmo2lOMFtAT3BlbkFJ9qwC52ExKO4FTTatmUNH',
    model='text-embedding-ada-002'
)

c:\Users\(Ai)AiSukmoren\Desktop\KMITL-present\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
pdf = PdfReader(r'C:\Users\(Ai)AiSukmoren\Desktop\KMITL-present\data\Tesla.pdf')

In [3]:
pdf = PdfReader(r'C:\Users\(Ai)AiSukmoren\Desktop\KMITL-present\data\Chap6.pdf')

In [4]:
# read text from pdf
raw_text = ''
for i, page in enumerate(pdf.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)
len(texts)

In [ ]:
document_search = FAISS.from_texts(texts, embedding)

In [ ]:
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = "when does Engineering Development	Program occur?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

In [ ]:
def process_pdf_to_faiss_search(pdf_path, embedding_model):
    # Read the PDF file
    pdf = PdfReader(pdf_path)
    
    # Extract text from the PDF
    raw_text = ''
    for page in pdf.pages:
        content = page.extract_text()
        if content:
            raw_text += content
    
    # Split the text using Character Text Splitter
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=800,
        chunk_overlap=200,
        length_function=len,
    )
    texts = text_splitter.split_text(raw_text)
    
    # Create FAISS document search
    document_search = FAISS.from_texts(texts, embedding_model)
    
    # Save the FAISS index locally
    document_search.save_local("faiss_index")

In [ ]:
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain

# Load the FAISS index with allow_dangerous_deserialization set to True
index_path = r"C:\Users\(Ai)AiSukmoren\Desktop\KMITL-present\faiss_index"
document_search = FAISS.load_local(index_path, embedding, allow_dangerous_deserialization=True)

# Define your query
query = "what the section olar Energy Offerings says"

# Perform a similarity search
docs = document_search.similarity_search(query)

# Load a QA chain
chain = load_qa_chain(llm, chain_type="stuff")

# Run the QA chain
answer = chain.run(input_documents=docs, question=query)

print("Answer:", answer)


In [ ]:
def gen_rag_answer(query:str)->str:
    # Load the FAISS index with allow_dangerous_deserialization set to True
    index_path = r"C:\Users\(Ai)AiSukmoren\Desktop\KMITL-present\faiss_index"
    document_search = FAISS.load_local(index_path, embedding, allow_dangerous_deserialization=True)
    
    # Perform a similarity search
    docs = document_search.similarity_search(query)

    # Load a QA chain
    chain = load_qa_chain(llm, chain_type="stuff")

    # Run the QA chain
    answer = chain.run(input_documents=docs, question=query)

    return answer

In [ ]:
res = gen_rag_answer('what the section olar Energy Offerings says')

In [ ]:
print(res)